**Bert model op basis van Bert van Universiteit van Groningen. Context handling moet nog aangepast worden. Oversampling a.d.h.v de mediaan. dynamisch treshhold zoeken voor unknown.**

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn.functional as F
import os 

In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_NoDupsLessThemes.xlsx")
df = pd.read_excel(file_path)

In [3]:
import re

# Drop unnecessary columns
columns_to_drop = ["context","file_name","question","statistical"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# ✅ Drop rare themes (appearing < 2 times)
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 100].index
df = df[df["theme"].isin(valid_themes)]

# ✅ Recompute label encoding AFTER filtering
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

# Amount of rows
print(f"Number of rows after filtering: {len(df)}")



Number of rows after filtering: 92503


In [4]:
print("All theme_ids:", sorted(df["theme_id"].unique()))
print("num_labels:", df["theme_id"].nunique())


All theme_ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
num_labels: 18


In [5]:
# ✅ 5. Split Data into Train & Test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["clean_text"].tolist(), df["theme_id"].tolist(), test_size=0.2, random_state=42, stratify=df["theme_id"]
)

In [6]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Create a small DataFrame from train lists
train_df = pd.DataFrame({
    "clean_text": train_texts,
    "theme_id": train_labels
})

# Compute class counts and use median as balancing target
theme_counts = train_df["theme_id"].value_counts()



from collections import Counter
print("Class distribution :", Counter(train_labels))


Class distribution : Counter({9: 20094, 2: 11404, 13: 8481, 10: 7612, 4: 6457, 6: 4601, 1: 2595, 8: 2543, 12: 2538, 14: 1588, 3: 1124, 11: 1018, 7: 990, 16: 896, 15: 826, 0: 619, 5: 418, 17: 198})


In [7]:
###### ✅ 7. Load BERT Tokenizer & Define Dataset Class
from transformers import RobertaTokenizer, RobertaForSequenceClassification
model_name = "pdelobelle/robbert-v2-dutch-base"  # pak juistee model
tokenizer = RobertaTokenizer.from_pretrained(model_name)

class ThemeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoding["labels"] = torch.tensor(self.labels[idx])
        return {key: val.squeeze(0) for key, val in encoding.items()}

train_dataset = ThemeDataset(train_texts, train_labels, tokenizer)
test_dataset = ThemeDataset(test_texts, test_labels, tokenizer)

In [8]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(
    model_name,
    num_labels=df["theme_id"].nunique()
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# ✅ Clip only the upper bound at 10
clipped_weights = np.clip(class_weights, None, 10.0)

# Convert to PyTorch tensor
class_weights_tensor = torch.tensor(clipped_weights, dtype=torch.float)


In [10]:
# Final check before training
print("✔️ Final weights used in training:")
for i, weight in enumerate(class_weights_tensor):
    print(f"Class {i} → {weight.item():.4f}")

✔️ Final weights used in training:
Class 0 → 6.6417
Class 1 → 1.5843
Class 2 → 0.3605
Class 3 → 3.6577
Class 4 → 0.6367
Class 5 → 9.8355
Class 6 → 0.8935
Class 7 → 4.1527
Class 8 → 1.6167
Class 9 → 0.2046
Class 10 → 0.5401
Class 11 → 4.0385
Class 12 → 1.6199
Class 13 → 0.4848
Class 14 → 2.5889
Class 15 → 4.9773
Class 16 → 4.5884
Class 17 → 10.0000


In [11]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=class_weights_tensor.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [12]:
# ✅ 9. Define Training Arguments (With Early Stopping)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ 11. Train Model with Early Stopping
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)

trainer.train()

C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.114800,1.604365,0.589374,0.631840,0.589374,0.600101
2,1.431700,1.394477,0.659640,0.678397,0.659640,0.664490
3,1.057500,1.349059,0.688071,0.709907,0.688071,0.693417
4,0.788200,1.393867,0.712070,0.724829,0.712070,0.715995
5,0.572100,1.504740,0.727258,0.736217,0.727258,0.729981
6,0.415800,1.706741,0.738771,0.742210,0.738771,0.739968
7,0.301900,1.815814,0.742771,0.747584,0.742771,0.744427
8,0.229200,1.918719,0.745311,0.747574,0.745311,0.746183


TrainOutput(global_step=74008, training_loss=0.8638946859788385, metrics={'train_runtime': 15957.803, 'train_samples_per_second': 37.099, 'train_steps_per_second': 4.638, 'total_flos': 1.5578833150446797e+17, 'train_loss': 0.8638946859788385, 'epoch': 8.0})

SAVE MODEL NAAR KUL DRIVE 

In [13]:
trainer.save_model("output/Robbert2")#naam


In [14]:
# Get predictions from the model
preds_output = trainer.predict(test_dataset)

# Extract logits (raw model outputs)
logits = preds_output.predictions  # shape: (num_samples, num_classes)

# Get predicted class labels
predictions = np.argmax(logits, axis=1)

# Save everything in a DataFrame
output_df = pd.DataFrame({
    "text": test_texts,
    "true_label": test_labels,
    "predicted_label": predictions,
    "logits": logits.tolist()  # Save logits for softmax + threshold later
})

# Save to CSV
output_df.to_csv("output/Robbert2.csv", index=False)#naam


In [15]:
from datetime import datetime

run_id = datetime.now().strftime("%Y-%m-%d_%H-%M")
save_path = rf"C:\Users\jefva\Documents\Master\Thesis_s2\results\mBERT\Run_{run_id}"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('C:\\Users\\jefva\\Documents\\Master\\Thesis_s2\\results\\mBERT\\Run_2025-04-16_20-30\\tokenizer_config.json',
 'C:\\Users\\jefva\\Documents\\Master\\Thesis_s2\\results\\mBERT\\Run_2025-04-16_20-30\\special_tokens_map.json',
 'C:\\Users\\jefva\\Documents\\Master\\Thesis_s2\\results\\mBERT\\Run_2025-04-16_20-30\\vocab.json',
 'C:\\Users\\jefva\\Documents\\Master\\Thesis_s2\\results\\mBERT\\Run_2025-04-16_20-30\\merges.txt',
 'C:\\Users\\jefva\\Documents\\Master\\Thesis_s2\\results\\mBERT\\Run_2025-04-16_20-30\\added_tokens.json')